In [21]:
import imageio
import sys
import numpy as np
import visvis as vv
from skimage import color, measure, morphology
from skimage.feature import peak_local_max
from scipy import ndimage
from skvideo.io import VideoCapture
import matplotlib.pyplot as plt
import pylab

In [3]:
def readvideo(src):
    return imageio.get_reader(src)

In [11]:
vid = imageio.get_reader('sailfin_3_300_F_Hillary_1_4_5_L.mpg', 'ffmpeg')

In [26]:
framix = np.linspace(1000,1004,1)
for num in framix:
    frame = vid.get_data(num)
    fig = pylab.figure()
    fig.suptitle('image #{}'.format(num), fontsize=20)
    pylab.imshow(frame)
pylab.show()

In [24]:
print frames[1000].shape

(1080, 1920, 3)


In [8]:
plt.figure()
plt.imshow(frames[2])

In [17]:
ls

HSVTrack.py*                         processData.py*
Untitled.ipynb                       processData_new.py*
getSailfinStats.py                   sailfin_3_300_F_Hillary_1_4_5_L.mpg
newtracker.py                        untitled
